In [1]:
E = 210000000
v=0.3
t=0.025
w=3000
import numpy as np

In [2]:

nodes = [(0,0), (0.5,0), (0.5,0.25), (0,0.25), (0.25,0.125)]
element_nodes=[(1,2,5),(2,3,5),(3,4,5),(4,1,5)] 
element_areas=[]
element_k = []
element_b = []
D = (E/(1-v**2)) * (np.array([1,v,0,v,1,0,0,0,(1-v)/2]).reshape((3,3)))
for element in element_nodes:
    x = []
    y = []
    for node in element:
        x.append(nodes[node-1][0])
        y.append(nodes[node-1][1])
    area_mat = np.array([[1,1,1],x,y])
    area = 0.5*np.linalg.det(area_mat)
    element_areas.append(area)
    
    b1 = y[1] - y[2]
    b2 = y[2] - y[0]
    b3 = y[0] - y[1]
    g1 = x[2] - x[1]
    g2 = x[0] - x[2]
    g3 = x[1] - x[0]
    
    B = np.array([[b1,0,b2,0,b3,0],[0,g1,0,g2,0,g3],[g1,b1,g2,b2,g3,b3]]) / (2*area)
    element_b.append(B)
    
    k = np.matmul(np.transpose(B),np.matmul(D,B))*area*t
    element_k.append(k)



In [3]:
K = np.zeros((len(nodes)*2,len(nodes)*2))
for node in range(len(element_nodes)):
    node_pos = []
    for i in element_nodes[node]:
        node_pos.append(i*2-2)
        node_pos.append(i*2-1)
    for i in range(len(node_pos)):
        K[node_pos[i]][node_pos] += np.transpose(element_k[node])[i]


In [5]:

U=np.zeros((len(nodes)*2,1))   
F=np.zeros((len(nodes)*2,1))   

F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0
F[8,0]=0
F[9,0]=0

Up=U[[2,3,4,5,8,9]]
Fp=F[[2,3,4,5,8,9]]
Kpp=K[[2,3,4,5,8,9]]
Kpp=Kpp[:,[2,3,4,5,8,9]]


Up=np.dot(np.linalg.inv(Kpp),Fp)
print("Nodal Displacements of [U2x U2y U3x U3y U5x U5y]' (in m)")
print(Up)

Nodal Displacements of [U2x U2y U3x U3y U5x U5y]' (in m)
[[ 6.92834261e-06]
 [ 7.13904355e-07]
 [ 6.92834261e-06]
 [-7.13904355e-07]
 [ 3.27082221e-06]
 [ 2.54109884e-21]]


In [6]:

U[[2,3,4,5,8,9]]=Up

F=np.dot(K,U)
print("Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y F5x F5y](in kN)")
print(F)

Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y F5x F5y](in kN)
[[-9.37500000e+00]
 [-3.75400427e+00]
 [ 9.37500000e+00]
 [-1.15463195e-14]
 [ 9.37500000e+00]
 [ 8.88178420e-15]
 [-9.37500000e+00]
 [ 3.75400427e+00]
 [-7.10542736e-15]
 [ 6.92838861e-15]]


In [7]:
u=[]
for node in element_nodes:
    u1 = np.asarray([U[node[0]*2-2],U[node[0]*2-1],U[node[1]*2-2],U[node[1]*2-1],U[node[2]*2-2],U[node[2]*2-1]])
    u.append(u1)


In [8]:
elem_stress_sigma=[]
for i in range(len(element_nodes)):
    temp = np.dot(D,np.dot(element_b[i],u[i]))
    elem_stress_sigma+=[temp]
elem_stress_sigma

[array([[3000.        ],
        [ 300.3203417 ],
        [  -9.61025093]]),
 array([[ 2.98077950e+03],
        [-3.05125467e+02],
        [-5.47313597e-13]]),
 array([[3000.        ],
        [ 300.3203417 ],
        [   9.61025093]]),
 array([[3.01922050e+03],
        [9.05766151e+02],
        [8.20970395e-13]])]

In [9]:
def principal_stress(sigma):
    R = (sigma[0] + sigma[1])/2
    Q = ((sigma[0] - sigma[1])/2)**2 + sigma[2]*sigma[2]
    M = 2*sigma[2]/(sigma[0] - sigma[1])
    s1 = R + np.sqrt(Q)
    s2 = R - np.sqrt(Q)
    theta = (np.arctan(M)/2)*180/np.pi
    return np.asarray([s1 , s2 , theta])

In [10]:
prin_stress = []
for sigma in elem_stress_sigma:
    prin_stress.append(principal_stress(sigma))
prin_stress

[array([[ 3.00003421e+03],
        [ 3.00286132e+02],
        [-2.03956612e-01]]),
 array([[ 2.98077950e+03],
        [-3.05125467e+02],
        [-9.54341635e-15]]),
 array([[3.00003421e+03],
        [3.00286132e+02],
        [2.03956612e-01]]),
 array([[3.01922050e+03],
        [9.05766151e+02],
        [2.22565199e-14]])]